In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/My Drive/AI/DL/fake-news/train.csv')

In [3]:
df = df.dropna()

In [4]:
x = df.drop('label',axis=1)


In [5]:
y = df['label']
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [6]:
x.shape, y.shape

((18285, 4), (18285,))

In [7]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout

In [9]:
voc_size = 5000

In [10]:
message = x.copy()

In [11]:
message.reset_index(inplace=True)

In [12]:
message

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
from nltk.stem.porter import PorterStemmer

In [16]:
ps = PorterStemmer()

In [17]:
corpus = []

In [18]:
for i in range(0,len(message)):
# for i in range(0,200):
  # print('Before : ', message['title'][i])
  review = re.sub('[^a-zA-Z]', ' ', message['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)
  # print('After : ', review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
one_hot_repr = [one_hot(word,voc_size) for word in corpus]
one_hot_repr

[[1541, 2105, 960, 1740, 3683, 1424, 966, 4623, 1461, 344],
 [4947, 1759, 299, 2416, 2411, 4170, 2289],
 [4289, 1108, 2108, 3261],
 [2188, 127, 2945, 720, 108, 4067],
 [4446, 2411, 3592, 2780, 3420, 3854, 2411, 1492, 2124, 2438],
 [2404,
  3737,
  3236,
  274,
  3510,
  1929,
  2444,
  38,
  4438,
  1727,
  3181,
  2256,
  4169,
  487,
  2289],
 [2480, 2810, 282, 3873, 2391, 4782, 3586, 3295, 3800, 432, 442],
 [4795, 1267, 631, 3521, 3396, 3415, 1929, 1849, 3800, 432, 442],
 [1841, 4830, 4637, 2211, 1933, 2953, 480, 1119, 1929, 1110],
 [1015, 4367, 3603, 1459, 3414, 3919, 4003, 1706],
 [1734, 4128, 1794, 1534, 2129, 714, 2433, 2256, 2266, 2464, 3412],
 [720, 4177, 3683, 2953, 1929, 3396],
 [1928, 2523, 4185, 188, 189, 2056, 1591, 4799, 2197],
 [1783, 37, 3422, 2319, 4006, 1705, 3549, 3800, 432, 442],
 [2722, 4193, 3441, 2417, 225, 3800, 432, 442],
 [738, 359, 305, 4110, 1421, 2251, 3719, 3282, 2344, 2204],
 [1706, 1321, 1759],
 [1488, 1207, 125, 607, 1929, 58, 3427, 2289],
 [2270, 2116

In [21]:
sent_len = 20
embeded_docs = pad_sequences(one_hot_repr, padding='pre',maxlen=sent_len)
embeded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1541,
       2105,  960, 1740, 3683, 1424,  966, 4623, 1461,  344], dtype=int32)

In [22]:
embedding_vector_feature = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=sent_len))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))



In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [25]:
import numpy as np
X_final = np.array(embeded_docs)
y_final = np.array(y)
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [27]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 2s 10ms/step - loss: 0.3362 - accuracy: 0.8384 - val_loss: 0.1931 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1462 - accuracy: 0.9431 - val_loss: 0.1896 - val_accuracy: 0.9226
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1088 - accuracy: 0.9596 - val_loss: 0.2100 - val_accuracy: 0.9221
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0876 - accuracy: 0.9698 - val_loss: 0.2261 - val_accuracy: 0.9153
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0757 - accuracy: 0.9740 - val_loss: 0.2896 - val_accuracy: 0.9122
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0577 - accuracy: 0.9809 - val_loss: 0.3424 - val_accuracy: 0.9132
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0453 - accuracy: 0.9852 - val_loss: 0.3603 - val_accuracy: 0.9145
Epoch

In [28]:
y_pred = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [29]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
confusion_matrix(y_test, y_pred)

array([[3112,  307],
       [ 228, 2388]])

In [32]:
from sklearn.metrics import accuracy_score

In [33]:
accuracy_score(y_test, y_pred)

0.9113504556752279